In [1]:
!pip install -q torchmetrics lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 6.2 MB/s eta 0:00:00


In [2]:
import torch
import lpips
import numpy as np
from PIL import Image
from torchmetrics.image import StructuralSimilarityIndexMeasure
from torchmetrics.multimodal.clip_score import CLIPScore
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import load_image
import os
from tqdm import tqdm

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


# Configuration

In [3]:
MODEL_PATH = "/content/drive/MyDrive/controlnet_model_output_v1"
BASE_MODEL = "runwayml/stable-diffusion-v1-5"

# Load Metrics

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# 1. SSIM (Higher is better, max 1.0)
ssim_metric = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)

# 2. LPIPS (Lower is better, min 0.0)
loss_fn_alex = lpips.LPIPS(net='alex').to(device)

# 3. CLIP Score (Higher is better)
# Measures consistency between Text Prompt and Image
clip_score_fn = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16").to(device)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


100%|██████████| 233M/233M [00:01<00:00, 199MB/s]


Loading model from: /usr/local/lib/python3.12/dist-packages/lpips/weights/v0.1/alex.pth


...

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

# Load Pipeline

In [5]:
print("Loading model...")
controlnet = ControlNetModel.from_pretrained(MODEL_PATH, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_MODEL, controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None
).to(device)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.set_progress_bar_config(disable=True)

Loading model...


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

merges.txt: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


# Load Dataset

In [6]:
import zipfile
import shutil
import json
from pathlib import Path

# --- CONFIGURATION ---
# UPDATE THIS PATH to where your zip file is in your Drive
ZIP_PATH = "/content/drive/MyDrive/ControlNet_Datasets/final_dataset_v4_backup.zip"
dataset_root = Path("/content/dataset")

# 1. Unzip the dataset to local Colab storage
if not dataset_root.exists():
    print(f"Unzipping {ZIP_PATH}...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(dataset_root)
    print("Unzip complete.")

# 2. Restructure for 'imagefolder' loading
# This function moves .jsonl files into the folders and renames them to metadata.jsonl
def prepare_folder(split_name):
    split_dir = dataset_root / split_name
    jsonl_source = dataset_root / f"{split_name}.jsonl"

    # If the jsonl is outside, move it in
    if jsonl_source.exists() and split_dir.exists():
        target_path = split_dir / "metadata.jsonl"
        shutil.move(str(jsonl_source), str(target_path))
        print(f"Moved {split_name}.jsonl to {target_path}")

    # If the jsonl was already inside but named wrong, rename it
    elif (split_dir / f"{split_name}.jsonl").exists():
        shutil.move(str(split_dir / f"{split_name}.jsonl"), str(split_dir / "metadata.jsonl"))
        print(f"Renamed {split_name}.jsonl inside {split_name} folder")

prepare_folder("train")
prepare_folder("validation")

print(f"Dataset ready at: {dataset_root}")

Unzipping /content/drive/MyDrive/ControlNet_Datasets/final_dataset_v4_backup.zip...
Unzip complete.
Moved train.jsonl to /content/dataset/train/metadata.jsonl
Moved validation.jsonl to /content/dataset/validation/metadata.jsonl
Dataset ready at: /content/dataset


In [7]:
import json
import os

def fix_metadata(split_name, folder_path):
    metadata_path = os.path.join(folder_path, split_name, "metadata.jsonl")

    if not os.path.exists(metadata_path):
        print(f"Skipping {split_name}: {metadata_path} not found.")
        return

    print(f"Fixing {split_name} metadata at {metadata_path}...")

    fixed_lines = []
    with open(metadata_path, 'r') as f:
        for line in f:
            data = json.loads(line)

            # 1. FIX: Rename 'image' to 'file_name'
            # The loader explicitly requires the key 'file_name'
            if "image" in data and "file_name" not in data:
                data["file_name"] = data.pop("image")

            # If your key is named something else (e.g. 'jpg'), change "image" above to that key.

            fixed_lines.append(json.dumps(data))

    with open(metadata_path, 'w') as f:
        f.write('\n'.join(fixed_lines))

    print(f"Success! {split_name} metadata updated.")

# Update this path to your actual dataset folder
DATA_DIR = "/content/dataset"

fix_metadata("train", DATA_DIR)
fix_metadata("validation", DATA_DIR)

Fixing train metadata at /content/dataset/train/metadata.jsonl...
Success! train metadata updated.
Fixing validation metadata at /content/dataset/validation/metadata.jsonl...
Success! validation metadata updated.


In [8]:
import json
import os

def fix_paths(split_name, folder_path):
    metadata_path = os.path.join(folder_path, split_name, "metadata.jsonl")

    if not os.path.exists(metadata_path):
        print(f"Skipping {split_name}: {metadata_path} not found.")
        return

    print(f"Fixing paths in {split_name}...")

    fixed_lines = []
    changes_count = 0

    with open(metadata_path, 'r') as f:
        for line in f:
            data = json.loads(line)

            # Check and fix 'file_name' (Target Image)
            # If path starts with 'train/' or 'validation/', remove it
            if "file_name" in data:
                original = data["file_name"]
                if original.startswith(f"{split_name}/"):
                    data["file_name"] = original.replace(f"{split_name}/", "", 1)
                    changes_count += 1

            # Check and fix 'conditioning_image' (Input Image)
            if "conditioning_image" in data:
                original = data["conditioning_image"]
                if original.startswith(f"{split_name}/"):
                    data["conditioning_image"] = original.replace(f"{split_name}/", "", 1)
                    changes_count += 1

            fixed_lines.append(json.dumps(data))

    # Overwrite the file with fixed paths
    with open(metadata_path, 'w') as f:
        f.write('\n'.join(fixed_lines))

    print(f"Done! Corrected {changes_count} paths in {split_name}.")

# Run the fix
DATA_DIR = "/content/dataset"
fix_paths("train", DATA_DIR)
fix_paths("validation", DATA_DIR)

Fixing paths in train...
Done! Corrected 20000 paths in train.
Fixing paths in validation...
Done! Corrected 10000 paths in validation.


In [9]:
import json
import os

# Check one entry to see if it works now
with open("/content/dataset/train/metadata.jsonl", 'r') as f:
    first_entry = json.loads(f.readline())

img_path = first_entry['file_name']
full_path = os.path.join("/content/dataset/train", img_path)

if os.path.exists(full_path):
    print(f"SUCCESS: Found image at {full_path}")
else:
    print(f"STILL FAILING: Could not find {full_path}")
    print("Please inspect your folder structure in the Files tab on the left.")

SUCCESS: Found image at /content/dataset/train/images/COCO_train2014_000000566583.jpg


In [10]:
file_path = '/content/dataset/validation/metadata.jsonl'

all_entries = []
with open(file_path, 'r') as f:
    for line in f:
        all_entries.append(json.loads(line))

# Ensure there are at least 50 entries to sample
# num_samples = min(5, len(all_entries))
# sampled_entries = random.sample(all_entries, num_samples)

# print(f"Successfully sampled {num_samples} entries.")
# for i, entry in enumerate(sampled_entries):
#     print(f"Sample {i+1}: {entry}")

# Evaluation Loop

In [11]:
scores = {"ssim": [], "lpips": [], "clip": []}

print(f"Starting evaluation on {len(all_entries)} samples...")

Starting evaluation on 5000 samples...


In [ ]:
import os

for sample in tqdm(all_entries):
    # 1. Load Inputs
    cond_image = load_image(os.path.join(DATA_DIR, 'validation', sample["conditioning_image"]))
    gt_image = load_image(os.path.join(DATA_DIR, 'validation', sample["file_name"])) # Resize to match gen output

    # 2. Generate Image
    generator = torch.manual_seed(42)
    generated_image = pipe(
        sample["text"],
        image=cond_image,
        num_inference_steps=20,
        generator=generator
    ).images[0]

    # # 3. Preprocess for Metrics (Convert to Tensors)
    # # Convert [0, 255] PIL -> [0.0, 1.0] Tensor (N, C, H, W)
    # gen_tensor = torch.from_numpy(np.array(generated_image)).float() / 255.0
    # gen_tensor = gen_tensor.permute(2, 0, 1).unsqueeze(0).to(device)

    # gt_tensor = torch.from_numpy(np.array(gt_image)).float() / 255.0
    # gt_tensor = gt_tensor.permute(2, 0, 1).unsqueeze(0).to(device)

    # --- PREPROCESSING START ---

    # A. For SSIM & LPIPS: Use Float [0.0, 1.0]
    # Convert PIL -> Numpy -> Tensor -> Normalize
    gen_array = np.array(generated_image)
    gt_array = np.array(gt_image)

    gen_tensor_norm = torch.from_numpy(gen_array).float() / 255.0
    gen_tensor_norm = gen_tensor_norm.permute(2, 0, 1).unsqueeze(0).to(device)

    gt_tensor_norm = torch.from_numpy(gt_array).float() / 255.0
    gt_tensor_norm = gt_tensor_norm.permute(2, 0, 1).unsqueeze(0).to(device)

    # B. For CLIP Score: Use Int [0, 255]
    # We pass the raw tensor (0-255) to avoid the "double rescaling" warning
    gen_tensor_raw = torch.from_numpy(gen_array).to(device)
    # CLIPScore expects (N, C, H, W) or (C, H, W) - usually handles permute automatically or expects channel first
    # TorchMetrics CLIP usually expects (N, C, H, W) in 0-255 range for best results with HF tokenizers
    gen_tensor_raw = gen_tensor_raw.permute(2, 0, 1).unsqueeze(0)

    # --- PREPROCESSING END ---

    # 4. Calculate SSIM (Uses Normalized Tensors)
    ssim_val = ssim_metric(gen_tensor_norm, gt_tensor_norm)
    scores["ssim"].append(ssim_val.item())

    # 5. Calculate LPIPS (Uses Normalized Tensors - Scaled to [-1, 1] internally or here)
    # LPIPS expects inputs in range [-1, 1]
    lpips_val = loss_fn_alex((gen_tensor_norm * 2) - 1, (gt_tensor_norm * 2) - 1)
    scores["lpips"].append(lpips_val.item())

    # 6. Calculate CLIP Score (Uses Raw Tensors to silence warning)
    clip_val = clip_score_fn(gen_tensor_raw, [sample["text"]])
    scores["clip"].append(clip_val.item())

# --- REPORT ---
print("\n=== Quantitative Analysis Results ===")
print(f"Average SSIM (Structure Consistency): {np.mean(scores['ssim']):.4f} (Higher is better)")
print(f"Average LPIPS (Perceptual Similarity): {np.mean(scores['lpips']):.4f} (Lower is better)")
print(f"Average CLIP Score (Text Consistency): {np.mean(scores['clip']):.4f} (Higher is better)")

 80%|████████  | 4005/5000 [1:34:11<23:33,  1.42s/it]